In [5]:
import sys
sys.path.insert(0, "../..")

import qlib
import pandas as pd
from qlib.config import REG_CN
from qlib.contrib.model.gbdt import LGBModel
from qlib.contrib.data.handler import Alpha158
from qlib.contrib.evaluate import (
    backtest as normal_backtest,
    risk_analysis,
)
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict


In [6]:
# use default data
provider_uri = "../../data/qlib_cn_stock"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

[18802:MainThread](2021-11-25 17:22:38,039) INFO - qlib.Initialization - [config.py:393] - default_conf: client.
[18802:MainThread](2021-11-25 17:22:38,042) WARNING - qlib.Initialization - [config.py:418] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[18802:MainThread](2021-11-25 17:22:38,045) INFO - qlib.Initialization - [__init__.py:57] - qlib successfully initialized based on client settings.
[18802:MainThread](2021-11-25 17:22:38,046) INFO - qlib.Initialization - [__init__.py:59] - data_path={'__DEFAULT_FREQ': PosixPath('/data3/xujianjin/qlib/data/qlib_cn_stock')}


In [7]:
hidden_sizes = [256]

# train model

In [8]:
market = "csi300"
benchmark = "SH000300"

###################################
# train model
###################################
data_handler_config = {
    "start_time": "2008-01-01",
    "end_time": "2020-08-01",
    "fit_start_time": "2008-01-01",
    "fit_end_time": "2014-12-31",
    "instruments": market,
}

task = {
    "model": {
        "class": "DNNModelPytorch",
        "module_path": "qlib.contrib.model.pytorch_nn",
        "kwargs": {
            "input_dim" : 158,
            "output_dim" : 1,
            "layers" : hidden_sizes,
            "lr" : 0.001,
            "max_steps" : 300,
            "batch_size" : 1024,
            "early_stop_rounds" : 50,
            "eval_steps" : 20,
            "lr_decay" : 0.96,
            "lr_decay_steps" : 100,
            "optimizer" : "adam",
            "loss" : "mse",
            "GPU" : 8,
            "seed" : None,
            "weight_decay" : 1e-4
        },
    },
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                "class": "Alpha158",
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,
            },
            "segments": {
                "train": ("2008-01-01", "2014-12-31"),
                "valid": ("2015-01-01", "2016-12-31"),
                "test": ("2017-01-01", "2020-08-01"),
            },
        },
    },
}

# model initiaiton
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])

# start exp to train model
with R.start(experiment_name="train_model"):
    R.log_params(**flatten_dict(task))
    model.fit(dataset)
    R.save_objects(trained_model=model)
    rid = R.get_recorder().id


[18802:MainThread](2021-11-25 17:22:38,229) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:71] - DNN pytorch version...
[18802:MainThread](2021-11-25 17:22:38,231) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:88] - DNN parameters setting:
layers : [256]
lr : 0.001
max_steps : 300
batch_size : 1024
early_stop_rounds : 50
eval_steps : 20
lr_decay : 0.96
lr_decay_steps : 100
optimizer : adam
loss_type : mse
eval_steps : 20
seed : None
device : cuda:8
use_GPU : True
weight_decay : 0.0001
[18802:MainThread](2021-11-25 17:22:38,237) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:132] - model:
Net(
  (dnn_layers): ModuleList(
    (0): Dropout(p=0.05, inplace=False)
    (1): Sequential(
      (0): Linear(in_features=158, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1)
    )
    (2): Dropout(p=0.05, inplace=False)
    (3): Linear(in_features=256, out_features=1, bias=True)
  )
)
[18802:

torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(5.4985, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(5.5046, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(5.8334, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(4.8776, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(6.5743, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cu

[18802:MainThread](2021-11-25 17:23:18,234) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:241] - [Epoch 20]: train_loss nan, valid_loss nan


torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(4.3265, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(6.2272, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(5.7700, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(6.9678, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(5.5331, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cu

[18802:MainThread](2021-11-25 17:23:18,431) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:241] - [Epoch 40]: train_loss nan, valid_loss nan
[18802:MainThread](2021-11-25 17:23:18,599) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:241] - [Epoch 60]: train_loss nan, valid_loss nan


...

[18802:MainThread](2021-11-25 17:23:20,251) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:241] - [Epoch 260]: train_loss nan, valid_loss nan
[18802:MainThread](2021-11-25 17:23:20,430) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:241] - [Epoch 280]: train_loss nan, valid_loss nan


torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(5.3430, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(5.7592, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(6.4959, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(5.3831, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(4.9628, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cu

[18802:MainThread](2021-11-25 17:23:20,610) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:241] - [Epoch 299]: train_loss nan, valid_loss nan
[18802:MainThread](2021-11-25 17:23:20,618) INFO - qlib.timer - [log.py:113] - Time cost: 0.002s | waiting `async_log` Done


torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(5.4078, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(6.9873, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(4.9397, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(6.6559, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cuda:8') tensor(nan, device='cuda:8', grad_fn=<MaxBackward1>) tensor(5.1848, device='cuda:8')
torch.Size([1024, 158]) torch.Size([1024, 1]) torch.Size([1024, 1]) tensor(nan, device='cu

EOFError: Ran out of input

# prediction, backtest & analysis

In [ ]:
###################################
# prediction, backtest & analysis
###################################
port_analysis_config = {
    "executor": {
        "class": "SimulatorExecutor",
        "module_path": "qlib.backtest.executor",
        "kwargs": {
            "time_per_step": "day",
            "generate_portfolio_metrics": True,
        },
    },
    "strategy": {
        "class": "TopkDropoutStrategy",
        "module_path": "qlib.contrib.strategy.signal_strategy",
        "kwargs": {
            "model": model,
            "dataset": dataset,
            "topk": 50,
            "n_drop": 5,
        },
    },
    "backtest": {
        "start_time": "2017-01-01",
        "end_time": "2020-08-01",
        "account": 100000000,
        "benchmark": benchmark,
        "exchange_kwargs": {
            "freq": "day",
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5,
        },
    },
}

# backtest and analysis
with R.start(experiment_name="backtest_analysis"):
    recorder = R.get_recorder(recorder_id=rid, experiment_name="train_model")
    model = recorder.load_object("trained_model")

    # prediction
    recorder = R.get_recorder()
    ba_rid = recorder.id
    sr = SignalRecord(model, dataset, recorder)
    sr.generate()

    # backtest & analysis
    par = PortAnaRecord(recorder, port_analysis_config, "day")
    par.generate()


# analyze graphs

In [ ]:
from qlib.contrib.report import analysis_model, analysis_position
from qlib.data import D
recorder = R.get_recorder(recorder_id=ba_rid, experiment_name="backtest_analysis")
print(recorder)
pred_df = recorder.load_object("pred.pkl")
pred_df_dates = pred_df.index.get_level_values(level='datetime')
report_normal_df = recorder.load_object("portfolio_analysis/report_normal_1day.pkl")
positions = recorder.load_object("portfolio_analysis/positions_normal_1day.pkl")
analysis_df = recorder.load_object("portfolio_analysis/port_analysis_1day.pkl")

## analysis position

### report

In [ ]:
analysis_position.report_graph(report_normal_df)

### risk analysis

In [ ]:
analysis_position.risk_analysis_graph(analysis_df, report_normal_df)

## analysis model

In [ ]:
label_df = dataset.prepare("test", col_set="label")
label_df.columns = ['label']

### score IC

In [ ]:
pred_label = pd.concat([label_df, pred_df], axis=1, sort=True).reindex(label_df.index)
analysis_position.score_ic_graph(pred_label)

### model performance

In [ ]:
analysis_model.model_performance_graph(pred_label)